In [1]:
data_train="newdataset/train"
data_val="newdataset/val"

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [3]:
image_size=(128, 128)
input_shape=(128,128,3)
num_classes=6

In [4]:
cnn_model=Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

In [5]:
cnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse+categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2
)

In [8]:


val_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory('newdataset\\train',
    target_size=image_size,
    batch_size=32,
    class_mode='sparse'
)

Found 2527 images belonging to 6 classes.


In [10]:
val_generator = val_datagen.flow_from_directory('newdataset//val',
    target_size=image_size,
    batch_size=32,
    class_mode='sparse'
)

Found 2527 images belonging to 6 classes.


In [11]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-5,
    verbose=1
)

In [12]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

In [13]:
cnn_model.compile(
    optimizer='adam',
    loss='sparse+categorical_crossentropy',  # ❌ THIS is incorrect
    metrics=['accuracy']
)


In [15]:

# history = cnn_model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=5,
#     validation_data=val_generator,
#     validation_steps=len(val_generator),
#     callbacks=[reduce_lr, early_stopping]#checkpoint
# )


cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # ✅ FIXED
    metrics=['accuracy']
)

history = cnn_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=2,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[reduce_lr, early_stopping]
)



Epoch 1/2
79/79 [==============================] - 99s 1s/step - loss: 1.5366 - accuracy: 0.4179 - val_loss: 3.7495 - val_accuracy: 0.2327 - lr: 0.0010
Epoch 2/2
79/79 [==============================] - 37s 473ms/step - loss: 1.4560 - accuracy: 0.4519 - val_loss: 4.0571 - val_accuracy: 0.3300 - lr: 0.0010
